In [1]:
from google.colab import files
uploaded = files.upload()

Saving labelled_train_set.csv to labelled_train_set.csv


In [2]:
import pandas as pd
df = pd.read_csv('labelled_train_set.csv')
df=df.dropna()
df.head()

,ID,Article,Type
0,TRAIN_1,There are a variety of emerging applications f...,Human-written
1,TRAIN_2,As each new means of communication and social ...,Human-written
2,TRAIN_3,"These suggestions include:, Learn about the pu...",Human-written
3,TRAIN_4,In recent years there has been growing concern...,Human-written
4,TRAIN_5,"By the late 1970s, researchers at many major u...",Human-written


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
import xgboost as xgb


In [4]:
import numpy as np
df['Type'] = np.where(df['Type'] == 'Human-written', 0, 1)


In [5]:
aitext=df[df['Type']==1]['Article']
humantext=df[df['Type']==0]['Article']

In [6]:
num_ai = len(aitext)
num_human = len(humantext)

labels = [1] * num_ai + [0] * num_human

texts = aitext + humantext

In [7]:
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(df['Article'])

In [10]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV
smote = SMOTE(random_state=42)

scale_pos_weight = len(df[df['Type'] == 0]) / len(df[df['Type'] == 1])

X_res, y_res = smote.fit_resample(X_train_tfidf, df['Type'])
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 300]
}

xgb_model = xgb.XGBClassifier(scale_pos_weight=scale_pos_weight,use_label_encoder=False, eval_metric='logloss')

grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, scoring='f1_macro', n_jobs=-1, verbose=2)
grid_search.fit(X_res, y_res)
best_params = grid_search.best_params_

best_model = xgb.XGBClassifier(**best_params, use_label_encoder=False, eval_metric='logloss')
best_model.fit(X_res, y_res)

Fitting 3 folds for each of 27 candidates, totalling 81 fits


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [13:24:24] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [13:24:32] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.2, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=3,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=300,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [11]:
from google.colab import files
uploaded = files.upload()

Saving unlabelled_test2.csv to unlabelled_test2.csv


In [12]:
df_test = pd.read_csv('unlabelled_test2.csv')
df_test.head()

,ID,Article
0,TEST_1,Grid computing is a type of distributed comput...
1,TEST_2,"RAID stands for ""Redundant Array of Independen..."
2,TEST_3,A computer password is a sequence of character...
3,TEST_4,For people who find commercial online services...
4,TEST_5,A Green PC is a computer that has been designe...


In [14]:
from sklearn.metrics import classification_report, accuracy_score
y_pred = best_model.predict(vectorizer.transform(df_test['Article']))

In [15]:
df_test['Type']=y_pred
df_test.head()

,ID,Article,Type
0,TEST_1,Grid computing is a type of distributed comput...,0
1,TEST_2,"RAID stands for ""Redundant Array of Independen...",1
2,TEST_3,A computer password is a sequence of character...,0
3,TEST_4,For people who find commercial online services...,0
4,TEST_5,A Green PC is a computer that has been designe...,1
